In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from snowflake.connector.pandas_tools import write_pandas
from sqlalchemy import create_engine
import snowflake.connector

d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\options.py:103: UserWarning: You have an incompatible version of 'pyarrow' installed (14.0.2), please install a version that adheres to: 'pyarrow<10.1.0,>=10.0.1; extra == "pandas"'
  warn_incompatible_dep(
Failed to import ArrowResult. No Apache Arrow result set format can be used. ImportError: DLL load failed while importing arrow_iterator: The specified procedure could not be found.


In [2]:
import snowflake.connector

In [3]:
## Set options to view all columns
pd.set_option('display.max_columns', None)

In [4]:
SNOWFLAKE_USER = 'kbharaj3'
SNOWFLAKE_PASSWORD = 'Snowfl@key0014'
SNOWFLAKE_ACCOUNT = 'qx25653.ca-central-1.aws'
SNOWFLAKE_WAREHOUSE = 'FOOTY_STORE'
SNOWFLAKE_DATABASE = 'GEGENSTATS'
SNOWFLAKE_SCHEMA = 'TABLES'

In [5]:
conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT,
    warehouse=SNOWFLAKE_WAREHOUSE,
    database=SNOWFLAKE_DATABASE,
    schema=SNOWFLAKE_SCHEMA
)

In [6]:
cursor = conn.cursor()

In [7]:
cursor.execute('SELECT * FROM TEAMS')
team_rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
team_names = pd.DataFrame(team_rows, columns=column_names)

In [8]:
cursor.execute('SELECT * FROM TEAM_STANDARD_STATS')
standard_rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
team_standard = pd.DataFrame(standard_rows, columns=column_names)

In [9]:
cursor.execute('SELECT * FROM TEAM_ATTACKING_STATS')
attacking_rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
team_attacking = pd.DataFrame(attacking_rows, columns=column_names)

In [10]:
cursor.execute('SELECT * FROM TEAM_DEFENDING_STATS')
defending_rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
team_defending = pd.DataFrame(defending_rows, columns=column_names)

In [11]:
cursor.execute('SELECT * FROM COMPETITIONS')
competition_rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
df_competitions = pd.DataFrame(competition_rows, columns=column_names)

In [12]:
team_names.drop(['TEAM_LOGO_URL'], axis=1, inplace=True)

In [13]:
team_standard = team_standard.merge(team_names, on='TEAM_FBREF_ID', how='left')
team_defending = team_defending.merge(team_names, on='TEAM_FBREF_ID', how='left')
team_attacking = team_attacking.merge(team_names, on='TEAM_FBREF_ID', how='left')

In [14]:
team_attacking = team_attacking.merge(team_standard, on=['TEAM_FBREF_ID','SEASON','TEAM_NAME','COMPETITION'], how='left')
team_defending = team_defending.merge(team_standard, on=['TEAM_FBREF_ID','SEASON','TEAM_NAME', 'COMPETITION'], how='left')

In [15]:
team_attacking = team_attacking.merge(df_competitions[['COMPETITION','COMPETITION_ACRONYM','SEASON']], 
                                      on=['COMPETITION','SEASON'], how='left')
team_defending = team_defending.merge(df_competitions[['COMPETITION','COMPETITION_ACRONYM','SEASON']], 
                                      on=['COMPETITION','SEASON'], how='left')

In [16]:
team_attacking['Goals/Game'] = team_attacking['GOALS_SCORED']/team_attacking['MATCHES_PLAYED']
team_attacking['Pass Completion (%)'] = team_attacking['PASS_COMPLETED']*100/team_attacking['PASS_ATTEMPTED']
team_attacking['Fouls Against/Game'] = team_attacking['FOULS_AGAINST']/team_attacking['MATCHES_PLAYED']
team_attacking['NPxG/Game'] = team_attacking['NPXG']/team_attacking['MATCHES_PLAYED']
team_attacking['Shots/Game'] = team_attacking['SHOTS']/team_attacking['MATCHES_PLAYED']
team_attacking['Shots On Target (%)'] = team_attacking['SHOTS_ON_TARGET']*100/team_attacking['SHOTS']
team_attacking['Take_ons Attempted/Game'] = team_attacking['TAKEONS_ATTEMPTED']/team_attacking['MATCHES_PLAYED']
team_attacking['Crosses into Pen Area'] = team_attacking['CROSSES_INTO_PA']

In [17]:
# team_defending['Clearances/Game'] = team_defending['CLEARANCES']/team_defending['MATCHES_PLAYED']
team_defending['Fouls Made/Game'] = team_defending['FOULS_MADE']/team_defending['MATCHES_PLAYED']
team_defending['Conceded/Game'] = team_defending['GOALS_CONCEDED']/team_defending['MATCHES_PLAYED']
team_defending['xG Against/Game'] = team_defending['XG_AGAINST']/team_defending['MATCHES_PLAYED']
# team_defending['Tackles Attempted/Game'] = team_defending['TACKLES']/team_defending['MATCHES_PLAYED']
team_defending['Tackles Won (%)'] = team_defending['TACKLES_WON']*100/team_defending['TACKLES']
# team_defending['Interceptions/Game'] = team_defending['INTERCEPTIONS']/team_defending['MATCHES_PLAYED']
# team_defending['Blocked Shots/Game'] = team_defending['BLOCKED_SHOTS']/team_defending['MATCHES_PLAYED']
team_defending['Possession Won'] = team_defending['POSS_WON']
team_defending['Opposition PPDA'] = team_defending['OPP_PPDA']
team_defending['Final 3rd Passes Against/Game'] = team_defending['FINAL_3RD_PASSES_AGAINST']/team_defending['MATCHES_PLAYED']
team_defending['Clean Sheets'] = team_defending['CLEAN_SHEETS']

In [18]:
team_attacking= team_attacking[['SEASON', 'TEAM_NAME','COMPETITION_ACRONYM' ,'Goals/Game', 'Pass Completion (%)', 'Fouls Against/Game', 'NPxG/Game', 
                                'Shots/Game', 'Shots On Target (%)', 'Take_ons Attempted/Game', 'Crosses into Pen Area']]


In [19]:
# team_defending= team_defending[['SEASON', 'TEAM_NAME','COMPETITION_ACRONYM' ,'Clearances/Game', 'Fouls Made/Game', 'Conceded/Game', 'xG Against/Game',
#                                 'Tackles Attempted/Game', 'Tackles Won (%)', 'Interceptions/Game', 'Blocked Shots/Game','Possession Won', 
#                                 'Opposition PPDA', 'Final 3rd Passes Against/Game', 'Clean Sheets']]

team_defending= team_defending[['SEASON', 'TEAM_NAME','COMPETITION_ACRONYM','MATCHES_PLAYED' ,'Fouls Made/Game', 'Conceded/Game', 
                                'xG Against/Game', 'Tackles Won (%)', 'Possession Won', 
                                'Opposition PPDA', 'Final 3rd Passes Against/Game', 'Clean Sheets']]

In [20]:
team_attacking_average = team_attacking.drop(columns=['TEAM_NAME']).groupby(['SEASON', 'COMPETITION_ACRONYM']).mean().reset_index()
team_attacking_average['TEAM_NAME'] = team_attacking_average['COMPETITION_ACRONYM'] +'_'+ team_attacking_average['SEASON'].astype(str) + '_Average'
team_attacking = pd.concat([team_attacking, team_attacking_average], ignore_index=True)

In [21]:
team_defending_average = team_defending.drop(columns=['TEAM_NAME']).groupby(['SEASON', 'COMPETITION_ACRONYM']).mean().reset_index()
team_defending_average['TEAM_NAME'] = team_defending_average['COMPETITION_ACRONYM'] +'_'+ team_defending_average['SEASON'].astype(str) + '_Average'
team_defending = pd.concat([team_defending, team_defending_average], ignore_index=True)

In [22]:
scaler = MinMaxScaler()
team_defending_scaled = (team_defending.drop(['TEAM_NAME','SEASON','COMPETITION_ACRONYM','MATCHES_PLAYED'], axis=1))
team_defending_scaled_1 = team_defending_scaled.drop(['Fouls Made/Game', 'Conceded/Game', 'xG Against/Game', 
                                                      'Final 3rd Passes Against/Game','Opposition PPDA'], axis=1)
team_defending_scaled_2 = team_defending_scaled[['Fouls Made/Game', 'Conceded/Game', 'xG Against/Game', 
                                                 'Final 3rd Passes Against/Game','Opposition PPDA']]
team_defending_scaled_1['Possession Won'] = team_defending_scaled_1['Possession Won']/team_defending['MATCHES_PLAYED']
team_defending_scaled_1['Clean Sheets'] = team_defending_scaled_1['Clean Sheets']/team_defending['MATCHES_PLAYED']
team_defending_scaled_1 = pd.DataFrame(scaler.fit_transform(team_defending_scaled_1), columns=team_defending_scaled_1.columns)

In [23]:
scaler = MinMaxScaler()
team_defending_scaled_2 = pd.DataFrame(scaler.fit_transform(team_defending_scaled_2), columns=team_defending_scaled_2.columns)
team_defending_scaled_2 = pd.DataFrame(scaler.fit_transform(1-team_defending_scaled_2), columns=team_defending_scaled_2.columns)
team_defending_scaled_2 = pd.DataFrame(scaler.inverse_transform(team_defending_scaled_2), columns=team_defending_scaled_2.columns)
team_defending_scaled = pd.concat([team_defending_scaled_1, team_defending_scaled_2], axis=1)

In [25]:
team_defending_scaled = pd.concat([team_defending[['TEAM_NAME','SEASON','COMPETITION_ACRONYM']], team_defending_scaled], axis=1)

In [26]:
# scaler = MinMaxScaler()
# team_defending_scaled = (team_defending.drop(['TEAM_NAME','SEASON'], axis=1))
# team_defending_scaled_1 = team_defending_scaled.drop(['Fouls Made/Game', 'Conceded/Game', 'xG Against/Game'], axis=1)
# team_defending_scaled_2 = team_defending_scaled[['Fouls Made/Game', 'Conceded/Game', 'xG Against/Game']]
# team_defending_scaled_1 = pd.DataFrame(scaler.fit_transform(team_defending_scaled_1), columns=team_defending_scaled_1.columns)
# scaler = MinMaxScaler()
# team_defending_scaled_2 = pd.DataFrame(scaler.fit_transform(team_defending_scaled_2), columns=team_defending_scaled_2.columns)
# team_defending_scaled_2 = pd.DataFrame(scaler.fit_transform(1-team_defending_scaled_2), columns=team_defending_scaled_2.columns)
# team_defending_scaled_2 = pd.DataFrame(scaler.inverse_transform(team_defending_scaled_2), columns=team_defending_scaled_2.columns)
# team_defending_scaled = pd.concat([team_defending_scaled_1, team_defending_scaled_2], axis=1)
# team_defending_scaled = pd.concat([team_defending[['TEAM_NAME','SEASON']], team_defending_scaled], axis=1)

In [27]:
scaler = MinMaxScaler()
team_attacking_scaled = (team_attacking.drop(['TEAM_NAME','SEASON','COMPETITION_ACRONYM'], axis=1))
team_attacking_scaled = pd.DataFrame(scaler.fit_transform(team_attacking_scaled), columns=team_attacking_scaled.columns)
team_attacking_scaled = pd.concat([team_attacking[['TEAM_NAME','SEASON','COMPETITION_ACRONYM']], team_attacking_scaled], axis=1)

In [28]:
team_attacking_radar_1 = team_attacking_scaled.melt(id_vars=["SEASON", "TEAM_NAME","COMPETITION_ACRONYM"]).sort_values(
                        by=["COMPETITION_ACRONYM","SEASON","TEAM_NAME"]).rename(columns={'value':'norm_value'})
team_attacking_radar_2 = team_attacking.melt(id_vars=["SEASON", "TEAM_NAME","COMPETITION_ACRONYM"]).sort_values(
                        by=["COMPETITION_ACRONYM","SEASON","TEAM_NAME"])

In [29]:
team_defending_radar_1 = team_defending_scaled.melt(id_vars=["SEASON", "TEAM_NAME","COMPETITION_ACRONYM"]).sort_values(
                        by=["COMPETITION_ACRONYM","SEASON","TEAM_NAME"]).rename(columns={'value':'norm_value'})
team_defending_radar_2 = team_defending.melt(id_vars=["SEASON", "TEAM_NAME","COMPETITION_ACRONYM"]).sort_values(
                        by=["COMPETITION_ACRONYM","SEASON","TEAM_NAME"])

In [30]:
team_defending_radar = team_defending_radar_1.merge(team_defending_radar_2, on=['SEASON','TEAM_NAME','COMPETITION_ACRONYM','variable'], how='left')
team_attacking_radar = team_attacking_radar_1.merge(team_attacking_radar_2, on=['SEASON','TEAM_NAME','COMPETITION_ACRONYM','variable'], how='left')

In [31]:
team_standard_radar_1 = team_attacking_scaled[['TEAM_NAME','SEASON','COMPETITION_ACRONYM','Pass Completion (%)','Goals/Game',
                                               'NPxG/Game','Shots/Game','Shots On Target (%)']]
team_standard_radar_2 = team_attacking[['TEAM_NAME','SEASON','COMPETITION_ACRONYM','Pass Completion (%)','Goals/Game','NPxG/Game',
                                               'Shots/Game','Shots On Target (%)']]
team_standard_radar_3 = team_defending_scaled[['TEAM_NAME','SEASON','COMPETITION_ACRONYM','Tackles Won (%)', 'Conceded/Game', 
                                               'xG Against/Game']]
team_standard_radar_4 = team_defending[['TEAM_NAME','SEASON','COMPETITION_ACRONYM','Tackles Won (%)', 'Conceded/Game', 
                                        'xG Against/Game']]

In [32]:
team_standard_radar_5 = team_standard_radar_1.merge(team_standard_radar_3, on=['TEAM_NAME','SEASON','COMPETITION_ACRONYM'], how='left')
team_standard_radar_6 = team_standard_radar_2.merge(team_standard_radar_4, on=['TEAM_NAME','SEASON','COMPETITION_ACRONYM'], how='left')

In [33]:
team_standard_radar_5 = team_standard_radar_5.melt(id_vars=["SEASON", "TEAM_NAME", "COMPETITION_ACRONYM"]).sort_values(
    by=["COMPETITION_ACRONYM","SEASON","TEAM_NAME"]).rename(columns={'value':'norm_value'})

In [34]:
team_standard_radar_6 = team_standard_radar_6.melt(id_vars=["SEASON", "TEAM_NAME", "COMPETITION_ACRONYM"]).sort_values(
    by=["COMPETITION_ACRONYM","SEASON","TEAM_NAME"])

In [35]:
team_standard_radar = team_standard_radar_5.merge(team_standard_radar_6, on=['SEASON','TEAM_NAME','COMPETITION_ACRONYM','variable'], how='left')

In [36]:
team_defending_radar.rename(columns={'variable':'VARIABLE','norm_value':"NORM_VALUE",
                                     'value':'VALUE'}, inplace=True)

In [37]:
team_attacking_radar.rename(columns={'variable':'VARIABLE','norm_value':"NORM_VALUE",
                                     'value':'VALUE'}, inplace=True)

In [38]:
team_standard_radar.rename(columns={'variable':'VARIABLE','norm_value':"NORM_VALUE",
                                     'value':'VALUE'}, inplace=True)

Upload to Snowflake now

In [39]:
SNOWFLAKE_USER = 'kbharaj3'
SNOWFLAKE_PASSWORD = 'Snowfl@key0014'
SNOWFLAKE_ACCOUNT = 'qx25653.ca-central-1.aws'
SNOWFLAKE_WAREHOUSE = 'FOOTY_STORE'
SNOWFLAKE_DATABASE = 'GEGENSTATS'
SNOWFLAKE_SCHEMA = 'RADAR_CHARTS'

In [40]:
conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT,
    warehouse=SNOWFLAKE_WAREHOUSE,
    database=SNOWFLAKE_DATABASE,
    schema=SNOWFLAKE_SCHEMA
    )

In [41]:
cur = conn.cursor()
cur.execute(f"USE WAREHOUSE {SNOWFLAKE_WAREHOUSE}")

In [42]:
create_schema_sql = f"CREATE SCHEMA IF NOT EXISTS {SNOWFLAKE_SCHEMA}"
cur.execute(create_schema_sql)

In [219]:
create_table_query = """
CREATE TABLE IF NOT EXISTS STANDARD_RADAR (
    SEASON INT,
    TEAM_NAME VARCHAR(255),
    COMPETITION_ACRONYM VARCHAR(255),
    VARIABLE VARCHAR(255),
    NORM_VALUE FLOAT,
    VALUE FLOAT,
    PRIMARY KEY (SEASON, COMPETITION_ACRONYM, TEAM_NAME, VARIABLE)
);
"""

In [220]:
cur.execute(create_table_query)

In [43]:
def insert_to_snowflake_DB(primary_keys, table_name, dataframe, replace=False):
    if replace:
        cur.execute(f"TRUNCATE TABLE {table_name}")
        success, nchunks, nrows, _ = write_pandas(conn, dataframe, table_name)
    else:
        cur.execute(f"SELECT {', '.join(primary_keys)} FROM {table_name}")
        existing_ids = cur.fetchall()
        existing_ids = [id for id in existing_ids]
        dataframe.set_index(primary_keys, inplace=True)
        new_rows = dataframe[~dataframe.index.isin(existing_ids)]
        new_rows.reset_index(inplace=True)
        dataframe.reset_index(inplace=True)
        if not new_rows.empty:
            success, nchunks, nrows, _ = write_pandas(conn, new_rows, table_name)
            print(f"Inserted {nrows} new rows")
        else:
            print("No new rows to insert")

In [50]:
insert_to_snowflake_DB(['SEASON','COMPETITION_ACRONYM','TEAM_NAME','VARIABLE'], 'STANDARD_RADAR', team_standard_radar, 
                       replace=True)

In [223]:
create_table_query = """
CREATE TABLE IF NOT EXISTS ATTACKING_RADAR (
    SEASON INT,
    TEAM_NAME VARCHAR(255),
    COMPETITION_ACRONYM VARCHAR(255),
    VARIABLE VARCHAR(255),
    NORM_VALUE FLOAT,
    VALUE FLOAT,
    PRIMARY KEY (SEASON, COMPETITION_ACRONYM, TEAM_NAME, VARIABLE)
);
"""

In [51]:
cur.execute(create_table_query)

NameError: name 'create_table_query' is not defined

In [52]:
insert_to_snowflake_DB(['SEASON','COMPETITION_ACRONYM','TEAM_NAME','VARIABLE'], 'ATTACKING_RADAR', team_attacking_radar, 
                       replace=True)

In [226]:
create_table_query = """
CREATE TABLE IF NOT EXISTS DEFENDING_RADAR (
    SEASON INT,
    TEAM_NAME VARCHAR(255),
    COMPETITION_ACRONYM VARCHAR(255),
    VARIABLE VARCHAR(255),
    NORM_VALUE FLOAT,
    VALUE FLOAT,
    PRIMARY KEY (SEASON, TEAM_NAME, VARIABLE)
);
"""

In [227]:
cur.execute(create_table_query)

In [46]:
insert_to_snowflake_DB(['SEASON','COMPETITION_ACRONYM','TEAM_NAME', 'VARIABLE'], 'DEFENDING_RADAR', team_defending_radar, 
                       replace=True)